In [1]:
from seleniumbase import Driver
import time
import pandas as pd


In [ ]:
# Configurar o driver com UC Mode
driver = Driver(uc=True, browser="chrome")

# Navegar até a página
driver.get("https://www.reclameaqui.com.br/empresa/nubank/lista-reclamacoes/")
time.sleep(5)  # Tempo pra carregar

# Rolar e clicar no botão "Últimas"
ultimas_button = driver.find_element("xpath", "//button[text()='Últimas']")
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", ultimas_button)
time.sleep(1)
driver.execute_script("arguments[0].click();", ultimas_button)
time.sleep(2)

# Configurar espera explícita
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
wait = WebDriverWait(driver, 30)

# Definir quantas páginas raspar
num_pages = 50  # Ajustado pra suas 50 páginas
data = []
previous_titles = set()

# Loop pra raspar as páginas
for page in range(num_pages):
    print(f"Raspando página {page + 1}")
    
    try:
        wait.until(EC.presence_of_element_located(("css selector", ".sc-1pe7b5t-0.eJgBOc")))
    except:
        print("Erro: Nenhuma div de reclamação encontrada.")
        break
    
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(2)
    
    complaint_divs = driver.find_elements("css selector", ".sc-1pe7b5t-0.eJgBOc")
    print(f"Encontrados {len(complaint_divs)} divs de reclamações.")
    
    current_titles = set()
    for div in complaint_divs:
        try:
            title = div.find_element("tag name", "h4").text
            description = div.find_element("tag name", "p").text
            data.append({"title": title, "description": description})
            current_titles.add(title)
            print(f"Reclamação capturada: {title}")
        except:
            print("Erro ao extrair uma reclamação")
    
    if current_titles == previous_titles and page > 0:
        print("Títulos são os mesmos")
    
    if page < num_pages - 1:
        try:
            next_button = wait.until(EC.element_to_be_clickable(("xpath", "//button[@aria-label='botão de navegação para a próxima página']")))
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(10)
            driver.refresh()
            time.sleep(5)
        except:
            print("Não foi possível avançar pra próxima página.")
            break
    
    previous_titles = current_titles

# Criar um DataFrame com os dados coletados
df = pd.DataFrame(data)

# Salvar o DataFrame em um arquivo CSV
df.to_csv("reclamacoes.csv", index=False, encoding="utf-8")

# Fechar o navegador
driver.quit()

print(f"Raspagem concluída! {len(data)} reclamações foram salvas em 'reclamacoes.csv'.")